In [ ]:
from sympy import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class BestPricing:
    
    def __init__(self, Qd, C, bC, N = None): 
        self.C = C # Cost per Quantity
        self.bC = bC # Base Cost
        self.Qd = Qd # Demand function with P as variable
        self.N = N # Product Produced (For step method)
    
    def FindMaximumInQone(self, Function, variable = 'x', check = False, N = None, QdVar = 'P',
                          QdFunc = None, PiFunc = None):
        # This section is to prepare functions to use for the step method to see if
        # the unit sold is more than number of product pruduced (For step method)
        Qdvar = str(QdVar)
        P = Symbol(QdVar)
        QdFuncLamb = lambdify(P, QdFunc)
        PiFuncLamb = lambdify(P, PiFunc)
        
        variable = str(variable)
        x = Symbol(variable)
        
        f = Function # Given function
        fprime = diff(f) # First derivative of the given function
        fprimeprime = diff(fprime) # Second derivative of the given function
        fx = lambdify(x, f)
        fprimex = lambdify(x, fprime)
        fprimeprimex = lambdify(x, fprimeprime)
        
        fprime_xaxis_intercept_x = solve(fprime) # List of all x values that are x-axis intercept of fprime     
                
        maximumcoordinate = [] # Index to represent if the vertex is maximum or not
        f_maximum_y = []
        fprime_xaxis_intercept_x_chosen = []
        f_maximum_y_chosen = []
        y_vertex = []
        
        # To remove the value of X (Price) in vertex that is negative
        for i in fprime_xaxis_intercept_x:
            if i < 0:
                fprime_xaxis_intercept_x.remove(i)
        
        # If there is no any vertex in Q1 
        if fprime_xaxis_intercept_x == []:
            return [0,0]
        
        # To get a list of value of Y (Net profit) in maximum point
        for i in fprime_xaxis_intercept_x:           
            f_maximum_y.append(fx(i))
        
        # To remove the value of Y (Net profit) in vertex that is negative
        for i in f_maximum_y:
            if i < 0:
                f_maximum_y.remove(i)
        
        # If there is no any vertex in Q1
        if f_maximum_y == []:
            return [0,0]
        
        # To tag each vertex if it is maximum(=1) or minimum(=0)
        for i in fprime_xaxis_intercept_x:
            if fprimeprimex(i) < 0:
                maximumcoordinate.append(1)
            else:
                maximumcoordinate.append(0)                
        
        # To get a list of index of vertex in other list that is maximum
        maximumcoordinate_numpy = np.array(maximumcoordinate)
        maximumcoordinate_tuple = np.where(maximumcoordinate_numpy == 1)
        maximumcoordinate_numpy = np.array(maximumcoordinate_tuple).flatten()
        maximumcoordinate = list(maximumcoordinate_numpy)
        
        # To choose the value of X (Price) in vertex that is maximum
        for i in maximumcoordinate:
            fprime_xaxis_intercept_x_chosen.append(fprime_xaxis_intercept_x[i])
        
        # To choose the value of Y (Net profit) in vertex that is maximum
        for i in maximumcoordinate:
            f_maximum_y_chosen.append(f_maximum_y[i])
        
        # To convert every maximum to array of coordinate as [x,y] in row 
        maximumQone_coordinate_array = np.array([fprime_xaxis_intercept_x_chosen,f_maximum_y_chosen]).T
        
        # To extract Y (Net profit) from maximumQone_coordinate_array
        for row in maximumQone_coordinate_array:
            y_vertex.append(row[1])
        
        # To get the index of the highest maximum (Highest value of Y)
        max_y = max(y_vertex)
        max_y_index = y_vertex.index(max_y)
        
        # To get only global maximum
        maximumQone_coordinate = maximumQone_coordinate_array[max_y_index]
        maximumQone_coordinate_list = list(maximumQone_coordinate.flatten()) # Index 0 as Price & 1 as AllProfit
        
        # Output for simple method
        if check == False:
            return maximumQone_coordinate_list
        
        # Output for step method to check if the product sold is less than product produced or not
        # If True return original answer
        # If False return the value of Price that will get demand equal to product produced
        # (Solve P from Demand Function) and return the value of Profit when sold with price of Price
        elif check == True:
            if QdFuncLamb(maximumQone_coordinate_list[0]) <= N:
                return maximumQone_coordinate_list
            elif QdFuncLamb(maximumQone_coordinate_list[0]) > N:
                return [solve(Qd-N, P)[-1], PiFuncLamb(solve(Qd-N, P)[-1])]
            
    def getBestPriceSimple(self, variable = 'P'):
        variable = str(variable)
        P = Symbol(variable) # Price
        pi = P - self.C # Profit per Product
        Q = self.Qd # Demand Function or Product Sold
        Pi = pi * Q - self.bC # Toltal Profit
        
        Fq = lambdify(P, Q)
        Coordinate = self.FindMaximumInQone(Pi ,P)
        
        BestPrice = Coordinate[0]
        Profit = Coordinate[1]
        UnitSold = Fq(BestPrice)
        Cost = self.C * UnitSold + self.bC
        Revenue = Profit + Cost
        
        return BestPrice, Profit, UnitSold, Cost, Revenue
            
    def getBestPriceStep(self, variable = 'P'):
        variable = str(variable)
        P = Symbol(variable) # Price
        C = self.C # Cost per Product
        N = self.N 
        bC = self.bC
        Q = self.Qd
        Fq = lambdify(P, Q)
        
        BestPrice = []
        Profit = []
        UnitSold = []
        Cost = []
        Revenue = []
                   
        for i, c in enumerate(C):
            n = N[i]
            pi = P - c # Profit per Quantity
            Pi = pi * Q - bC # Total Profit
            Coordinate = self.FindMaximumInQone(Pi, P, check = True, N = n, QdFunc = Q, PiFunc = Pi)
            
            
            BestPrice.append(Coordinate[0])
            Profit.append(Coordinate[1] - (n - Fq(Coordinate[0])) * c)
            UnitSold.append(Fq(Coordinate[0]))
            Cost.append(c * n + bC)
            Revenue.append((Coordinate[1] - (n - Fq(Coordinate[0])) * c) + (c * n + bC))
        
        max_Profit = max(Profit)
        max_Profit_index = Profit.index(max_Profit)
            
        Max_BestPrice = BestPrice[max_Profit_index]
        Max_Profit = Profit[max_Profit_index]
        Max_UnitSold = UnitSold[max_Profit_index]
        Max_Cost = Cost[max_Profit_index]
        Max_Revenue = Revenue[max_Profit_index]
        Max_N = N[max_Profit_index]
            
        return Max_BestPrice, Max_Profit, Max_UnitSold, Max_Cost, Max_Revenue, Max_N            

In [ ]:
P = Symbol('P')
Qd = 1500-P/2 #Demand Function
C = 1200 #Variable Cost
bC = 2600 #Fixed Cost

Model = BestPricing(Qd, C, bC)

In [ ]:
BestPrice, Profit, UnitSold, Cost, Revenue = Model.getBestPriceSimple()
[BestPrice, Profit, UnitSold, Cost, Revenue]

[2100, 402400.000000000, 450.000000000000, 542600.000000000, 945000.000000000]

In [ ]:
P = Symbol('P')
Qd = 1500-P/2 #Demand Function
C = [1200,1000,800] #Variable Cost
bC = 2600 #Fixed Cost
N = [440,450,500] #จำนวนสินค้าที่ต้องผลิต

Model = BestPricing(Qd, C, bC, N)

In [ ]:
BestPrice, Profit, UnitSold, Cost, Revenue, N = Model.getBestPriceStep()
[BestPrice, Profit, UnitSold, Cost, Revenue,N]

[2000, 597400.000000000, 500.000000000000, 402600, 1000000.00000000, 500]